In [ ]:
import os
import pandas as pd

train_path = '/content/drive/MyDrive/Datavidia/train.csv'
test_path = '/content/drive/MyDrive/Datavidia/test.csv'

train_df = pd.read_csv(train_path)
train_df

,review_id,review_text,category
0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,"Penginapannya rapi, bersih, suasana tenang, pe...",1
1,7d8cb2e465c5602e51c1ba8710e8ff16,"wifi tidak sampai kamar, tidak tersedia heater...",0
2,1f4447c73e5e1aac462d05b0ecbd5a55,"Lantai kamar mandi kotor, bau, dan seperti sdh...",0
3,b9aefbfb0debbfff43f8b268c82b43c0,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",0
4,e2afef18bd695fae43d4d53b8c547bd4,bocor lagi,0
...,...,...,...
14851,5ed96732e4f1e1e69dfb8669cf507587,kotor dan tidak ada wifi,0
14852,b0692bdf4926ee0d58c2d9a0f28b0d62,Kamar tidak bersih,0
14853,5217f5f284066f232d7b3c3bbd3cccd2,kamarnya terlalu kecil beberapa cm antara kasu...,0
14854,31ca5a04ccc0dae4eaafc91c9822bede,Wifinya sangat kurang memadai..,0


In [ ]:
test_df = pd.read_csv(test_path)
test_df

,review_id,review_text
0,7302180ac7160f04a405d8aa7bd6beb8,"Kasur rusak, punggung saya sakit, kasurnya ga ..."
1,3184c670e170f565c7522eb76a320ba1,gerah
2,e33abf6bb5d5a9a77c339043b1725dc8,"tempat ridur atas bawah.. ac nya pararel, tdk ..."
3,061d388950340070a6ac03fabc9027b0,"ac kurang dingin, tidak ada snack sperti di foto"
4,e62d87c348674b6c06856964f3ab16b8,Lumayan untuk guest house dengan standart harg...
...,...,...
3709,4fac4c272d9194021f921138ebd5b389,Tolong kamar 214 kulkasnya dibagusin..!!!
3710,4fabd4722dd4074841f9656f75105f09,"buruk sekali, sangat mengecewakan.\n"
3711,44ff89997547d54cccb2a33776181240,Lampu redup. Bau pengap. Dalam kamar mandi ga ...
3712,c46cf84e3665a10199a539fee919edd6,"Kamar mawar tidak bisa dikunci dari luar, suas..."


In [ ]:
import numpy as np

train_array = train_df.values
print(train_array.shape)
test_array = test_df.values
print(test_array.shape)

(14856, 3)
(3714, 2)


In [ ]:
def make_lower(data):
  temp = data
  for i in temp:
    # print(i)
    i[1] = i[1].lower()
  return temp

train_array = make_lower(train_array)
test_array = make_lower(test_array)

In [ ]:
import re

def delete_specialchar(data):
  temp = data
  for i in temp:
    i[1] = i[1].replace('\n', ' ')
    for j in i[1].split("\n"):
      i[1] = re.sub(r"[^a-zA-Z0-9]+", ' ', j)
    i[1] = i[1].replace('  ', ' ')
    i[1] = i[1].replace('   ', ' ')
  return temp
  
train_array2 = delete_specialchar(train_array)
test_array2 = delete_specialchar(test_array)

In [ ]:
def all_words(data):
  word_banks = []
  for i in data:
    for j in i[1].split(' '):
      if j not in word_banks:
        word_banks.append(j)
  return word_banks

train_all_words = sorted(all_words(train_array2))
test_all_words = sorted(all_words(test_array2))
print('train words count :',len(train_all_words))
print('test words count :',len(test_all_words))

train words count : 13663
test words count : 6439


In [ ]:
all_words = sorted(train_all_words + test_all_words)
all_words = [x.lower() for x in all_words]
print('All joined words \t:',len(all_words))
# print(all_words)
all_words_unique = list(dict.fromkeys(all_words))
all_words_unique.remove('')
print('All joined words unique :',len(all_words_unique))
print(all_words_unique)
all_columns = ['main-id','main-label'] + all_words_unique
print(all_columns)
all_columns2 = ['main-id'] + all_words_unique
print(all_columns2)

All joined words 	: 20102
All joined words unique : 15408
['0', '00', '000', '002', '006', '00an', '00wib', '00wita', '01', '02', '024', '03', '04', '05', '06', '0631', '07', '08', '087777926906', '09', '1', '10', '100', '1000', '1000x', '100k', '100rb', '100ribu', '100ribuan', '101', '102', '103', '104', '105', '106', '107', '108', '109', '10an', '10kamar', '10malam', '10menit', '10rb', '10x', '11', '110', '111', '112b', '113', '114', '115', '116', '117', '11malam', '11rb', '12', '120', '120k', '120x160', '121', '122', '125rb', '126', '129', '12an', '12malam', '12mlm', '12saya', '13', '1300', '130rb', '131rb', '13april', '14', '1400', '143', '145rban', '148rb', '15', '150', '150k', '150rb', '151', '154', '156', '15menit', '15mnt', '16', '160an', '160rb', '165rb', '16c', '16derajat', '17', '170', '17072710891541', '170cm', '171ribu', '175', '179rb', '17c', '18', '1807310155114', '180cm', '19', '190an', '195', '1am', '1b', '1botol', '1bulan', '1bungkus', '1hidup', '1jam', '1kali', '1kam

In [ ]:
import sys
import time
import decimal

def make_dataset_train(data, awu):
  temp = data
  y = []
  start = 1
  count = len(temp)
  inittime = time.time()
  for index,i in enumerate(temp):
    #progress
    d = round(decimal.Decimal(100*start/count), 2)
    c = round(decimal.Decimal(start/(time.time()-inittime)), 2)
    eta = round(decimal.Decimal(int((count-start)/c)), 2)
    mins, secs = divmod(eta, 60)
    hours , mins = divmod(mins, 60)
    if secs < 10:
      secs = '0'+str(int(secs))
    else:
      secs = str(int(secs))
    if mins < 10:
      mins = '0'+str(mins)
    else:
      mins = str(mins)
    if hours < 10:
      hours = '0'+str(hours)
    else:
      hours = str(hours)
    b = str(start) + ' of ' + str(count) + ' data ( '+ str(d) + '% '+str(c)+'file/s ETA: '+hours+':'+mins+':'+secs+' ) ' + 'data_id : '+i[0]
    sys.stdout.write('\r'+b)
    start += 1
    y.append([])
    y[index].append(i[0])
    y[index].append(i[2])
    for index2,j in enumerate(awu):
      if j in i[1].split(' '):
        temp2 = sum(j == s for s in i[1].split(' '))
        y[index].append(temp2)
      else:
        y[index].append(0)
  return y

train_dataset = pd.DataFrame(make_dataset_train(train_array2, all_words_unique), columns=all_columns)
train_dataset

14856 of 14856 data ( 100.00% 69.67file/s ETA: 00:00:00 ) data_id : aadb614310e5ceb79b4c68d14f8dab8a

,main-id,main-label,0,00,000,002,006,00an,00wib,00wita,01,02,024,03,04,05,06,0631,07,08,087777926906,09,1,10,100,1000,1000x,100k,100rb,100ribu,100ribuan,101,102,103,104,105,106,107,108,109,...,years,yee,yess,yet,yg,ygdi,yh,yng,yo,yogya,yogyakarta,yogyatorium,you,young,your,yourself,youtu,youtube,yrap,yth,yummy,yuyu,yy,z,zaman,zaza,zein,zenroom,zenrooms,zia,zih,zizi,zone,zonk,zoo,zook,zooooonnnkkk,zuhur,zz,zzz
0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7d8cb2e465c5602e51c1ba8710e8ff16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1f4447c73e5e1aac462d05b0ecbd5a55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,b9aefbfb0debbfff43f8b268c82b43c0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,e2afef18bd695fae43d4d53b8c547bd4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14851,5ed96732e4f1e1e69dfb8669cf507587,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14852,b0692bdf4926ee0d58c2d9a0f28b0d62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14853,5217f5f284066f232d7b3c3bbd3cccd2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14854,31ca5a04ccc0dae4eaafc91c9822bede,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_dataset.to_csv('/content/drive/MyDrive/Datavidia/train_dataset_advanced.csv')

### Test Section

In [ ]:
#Test data
def make_dataset_test(data, awu):
  temp = data
  y = []
  start = 1
  count = len(temp)
  inittime = time.time()
  for index,i in enumerate(temp):
    #progress
    d = round(decimal.Decimal(100*start/count), 2)
    c = round(decimal.Decimal(start/(time.time()-inittime)), 2)
    eta = round(decimal.Decimal(int((count-start)/c)), 2)
    mins, secs = divmod(eta, 60)
    hours , mins = divmod(mins, 60)
    if secs < 10:
      secs = '0'+str(int(secs))
    else:
      secs = str(int(secs))
    if mins < 10:
      mins = '0'+str(mins)
    else:
      mins = str(mins)
    if hours < 10:
      hours = '0'+str(hours)
    else:
      hours = str(hours)
    b = str(start) + ' of ' + str(count) + ' data ( '+ str(d) + '% '+str(c)+'file/s ETA: '+hours+':'+mins+':'+secs+' ) ' + 'data_id : '+i[0]
    sys.stdout.write('\r'+b)
    start += 1
    y.append([])
    y[index].append(i[0])
    for index2,j in enumerate(awu):
      if j in i[1].split(' '):
        temp2 = sum(j == s for s in i[1].split(' '))
        y[index].append(temp2)
      else:
        y[index].append(0)
  return y

test_dataset = pd.DataFrame(make_dataset_test(test_array2, all_words_unique), columns=all_columns2)
test_dataset

3714 of 3714 data ( 100.00% 72.29file/s ETA: 00:00:00 ) data_id : 82d1677f2aa50eb79fb4e6f496384ef2

,main-id,0,00,000,002,006,00an,00wib,00wita,01,02,024,03,04,05,06,0631,07,08,087777926906,09,1,10,100,1000,1000x,100k,100rb,100ribu,100ribuan,101,102,103,104,105,106,107,108,109,10an,...,years,yee,yess,yet,yg,ygdi,yh,yng,yo,yogya,yogyakarta,yogyatorium,you,young,your,yourself,youtu,youtube,yrap,yth,yummy,yuyu,yy,z,zaman,zaza,zein,zenroom,zenrooms,zia,zih,zizi,zone,zonk,zoo,zook,zooooonnnkkk,zuhur,zz,zzz
0,7302180ac7160f04a405d8aa7bd6beb8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3184c670e170f565c7522eb76a320ba1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,e33abf6bb5d5a9a77c339043b1725dc8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,061d388950340070a6ac03fabc9027b0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,e62d87c348674b6c06856964f3ab16b8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3709,4fac4c272d9194021f921138ebd5b389,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3710,4fabd4722dd4074841f9656f75105f09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3711,44ff89997547d54cccb2a33776181240,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3712,c46cf84e3665a10199a539fee919edd6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_dataset.to_csv('/content/drive/MyDrive/Datavidia/test_dataset_advanced.csv')

In [30]:
import pandas as pd

test_dataset = pd.read_csv('/content/drive/MyDrive/Datavidia/test_dataset_advanced.csv', index_col=0) 
main_label = pd.read_csv('/content/drive/MyDrive/Datavidia/best_result_downloaded.csv')['category'].values

In [31]:
test_dataset.insert(1, 'main-label', main_label)

In [32]:
test_dataset

,main-id,main-label,0,00,000,002,006,00an,00wib,00wita,01,02,024,03,04,05,06,0631,07,08,087777926906,09,1,10,100,1000,1000x,100k,100rb,100ribu,100ribuan,101,102,103,104,105,106,107,108,109,...,years,yee,yess,yet,yg,ygdi,yh,yng,yo,yogya,yogyakarta,yogyatorium,you,young,your,yourself,youtu,youtube,yrap,yth,yummy,yuyu,yy,z,zaman,zaza,zein,zenroom,zenrooms,zia,zih,zizi,zone,zonk,zoo,zook,zooooonnnkkk,zuhur,zz,zzz
0,7302180ac7160f04a405d8aa7bd6beb8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3184c670e170f565c7522eb76a320ba1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,e33abf6bb5d5a9a77c339043b1725dc8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,061d388950340070a6ac03fabc9027b0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,e62d87c348674b6c06856964f3ab16b8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3709,4fac4c272d9194021f921138ebd5b389,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3710,4fabd4722dd4074841f9656f75105f09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3711,44ff89997547d54cccb2a33776181240,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3712,c46cf84e3665a10199a539fee919edd6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
train_dataset = pd.read_csv('/content/drive/MyDrive/Datavidia/train_dataset_advanced.csv', index_col=0) )

(14856, 15410)
                               main-id  main-label  0  ...  zuhur  zz  zzz
0     7302180ac7160f04a405d8aa7bd6beb8           0  0  ...      0   0    0
1     3184c670e170f565c7522eb76a320ba1           0  0  ...      0   0    0
2     e33abf6bb5d5a9a77c339043b1725dc8           0  0  ...      0   0    0
3     061d388950340070a6ac03fabc9027b0           0  0  ...      0   0    0
4     e62d87c348674b6c06856964f3ab16b8           1  0  ...      0   0    0
...                                ...         ... ..  ...    ...  ..  ...
3709  4fac4c272d9194021f921138ebd5b389           0  0  ...      0   0    0
3710  4fabd4722dd4074841f9656f75105f09           0  0  ...      0   0    0
3711  44ff89997547d54cccb2a33776181240           0  0  ...      0   0    0
3712  c46cf84e3665a10199a539fee919edd6           0  0  ...      0   0    0
3713  82d1677f2aa50eb79fb4e6f496384ef2           1  0  ...      0   0    0

[3714 rows x 15410 columns]


In [38]:
print(train_dataset.shape)
print(test_dataset.shape)
train_dataset_new = train_dataset.append(test_dataset, ignore_index=True)
train_dataset_new

(14856, 15410)
(3714, 15410)


,main-id,main-label,0,00,000,002,006,00an,00wib,00wita,01,02,024,03,04,05,06,0631,07,08,087777926906,09,1,10,100,1000,1000x,100k,100rb,100ribu,100ribuan,101,102,103,104,105,106,107,108,109,...,years,yee,yess,yet,yg,ygdi,yh,yng,yo,yogya,yogyakarta,yogyatorium,you,young,your,yourself,youtu,youtube,yrap,yth,yummy,yuyu,yy,z,zaman,zaza,zein,zenroom,zenrooms,zia,zih,zizi,zone,zonk,zoo,zook,zooooonnnkkk,zuhur,zz,zzz
0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7d8cb2e465c5602e51c1ba8710e8ff16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1f4447c73e5e1aac462d05b0ecbd5a55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,b9aefbfb0debbfff43f8b268c82b43c0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,e2afef18bd695fae43d4d53b8c547bd4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18565,4fac4c272d9194021f921138ebd5b389,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18566,4fabd4722dd4074841f9656f75105f09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18567,44ff89997547d54cccb2a33776181240,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18568,c46cf84e3665a10199a539fee919edd6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
train_dataset_new.to_csv('/content/drive/MyDrive/Datavidia/train_dataset_advanced_large.csv')